In [1]:
import pandas as pd
import csv
import yfinance as yf

#Load Data

#Insider Data Amazon
insider_df = pd.read_csv('C:/Users/trist/code/Data/Finviz_insider/Data/AMZN.csv')

# Stock Data Amazon
ticker_symbol = 'AMZN'
start_date = '2023-01-01'
end_date = '2023-11-30'
amazon_df = yf.download(ticker_symbol, start=start_date, end=end_date)

# insider_df: name='Date', dtype= 'object'
# amazon_df: name='Date', dtype='datetime64[ns]' 


[*********************100%%**********************]  1 of 1 completed


### Todo
Diskutieren ob percentage_change = (daily_change['Close'].iloc[-1] - row['Open']) / row['Open'] * 100 Sinn macht. Vielleicht besser an Tradeday die 'Open' col zu verwenden?

wenn das modell jeden Tag läuft, dann schauen, dass ich das auf Minutenbasis umstelle. yfinance stellt Daten auf minutenbasis bis zu 30 Tage in die Vergangenheit zur Verfügung.

In [2]:
#Enrich df with weekdays and ticker
from datetime import datetime

insider_df['SEC_date'] = pd.to_datetime(insider_df['SEC_date'], format='%b %d %I:%M %p')
insider_df['Date'] = insider_df['SEC_date'].dt.strftime('%d.%m.%Y')
current_year = datetime.now().year
insider_df['Date'] = pd.to_datetime(insider_df['Date'], format='%d.%m.%Y')
insider_df['Date'] = insider_df['Date'].apply(lambda x: x.replace(year=current_year))
insider_df['Weekday_insider_trade'] = insider_df['Date'].dt.day_name()
insider_df['Time'] = insider_df['SEC_date'].dt.strftime('%H:%M')

amazon_df['Weekday_stocks'] = amazon_df.index.day_name()
amazon_df['Ticker'] = ticker_symbol


In [3]:
#Merge
insider_df = insider_df.sort_values('Date')
amazon_df = amazon_df.sort_values('Date')

merged_df = pd.merge_asof(insider_df, amazon_df, on='Date', direction='forward')

merged_df

In [39]:
#Creating KPIs
small_df = merged_df[['Date', 'Weekday_insider_trade', 'Time', 'insider', 'Open', 'Close', 'Weekday_stocks']]
stock_df_copy = small_df.copy()

# Calculate the daily percentage change using .loc on the copied DataFrame

####Baustelle##############
#stock_df_copy.loc[:, 'DailyChange'] = (stock_df_copy['Close'] - stock_df_copy['Open']) / stock_df_copy['Open'] * 100

### Calculate a 3, 7, 30 day percentage change ###
stock_df_copy['3DayChange'] = 0.0
stock_df_copy['WeekAfterChange'] = 0.0
stock_df_copy['MonthAfterChange'] = 0.0

counter = 0  # Initialize the counter

from pandas.tseries.offsets import BDay
from datetime import timedelta


for index, row in stock_df_copy.iterrows():
    insidertrade_date = row['Date']
    insidertrade_time = row['Time']
    insidertrade_weekday = row['Weekday_insider_trade']

    insidertrade_time = datetime.strptime(insidertrade_time, "%H:%M")
    threshold_time = datetime.strptime("17:00", "%H:%M")
    
    print(insidertrade_date)
    print(insidertrade_weekday)


    #falls Formular nach Börsenschluss hochgeladen wird
    if insidertrade_time > threshold_time:
        insidertrade_date = insidertrade_date + BDay(1)

    print("Nach if:" + str(insidertrade_date))
    print("\n")

    #Update DailyChange Col
    daily_change = amazon_df[amazon_df.index == insidertrade_date]
    percentage_change = (daily_change['Close'].iloc[-1] - row['Open']) / row['Open'] * 100
    stock_df_copy.at[index, 'DailyChange'] = percentage_change
    
    
    #Update 3DayChange Col
    threedays_after_date = insidertrade_date + BDay(3)  # Use business day offset to add 3 weekdays    
    threedays_after_date = amazon_df[(amazon_df.index >= insidertrade_date) & (amazon_df.index <= threedays_after_date)]

    
    if len(threedays_after_date) < 3: 
        stock_df_copy.at[index, '3DayChange'] = None
    else:
        percentage_change = (threedays_after_date['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
        stock_df_copy.at[index, '3DayChange'] = percentage_change
   

    #Update WeekAfterChange Col
    week_after_date = insidertrade_date + BDay(7)
    week_after_data = amazon_df[(amazon_df.index >= insidertrade_date) & (amazon_df.index <= week_after_date)]
    if len(week_after_data) < 7:
        stock_df_copy.at[index, 'WeekAfterChange'] = None
    else:
        percentage_change = (week_after_data['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
        stock_df_copy.at[index, 'WeekAfterChange'] = percentage_change


    #Update MonthAfterChange Col
    Month_After_Change = insidertrade_date + BDay(30)
    Month_After_Change = amazon_df[(amazon_df.index >= insidertrade_date) & (amazon_df.index <= Month_After_Change)]
    

    if len(Month_After_Change) < 30:
        stock_df_copy.at[index, 'MonthAfterChange'] = None
    else:
        percentage_change = (Month_After_Change['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
        stock_df_copy.at[index, 'MonthAfterChange'] = percentage_change

    counter +=1
    if counter ==6:
        break

#stock_df_copy.head(50)

2023-03-23 00:00:00
Thursday
Nach if:2023-03-23 00:00:00


2023-04-05 00:00:00
Wednesday
Nach if:2023-04-05 00:00:00


2023-04-06 00:00:00
Thursday
Nach if:2023-04-06 00:00:00


2023-04-06 00:00:00
Thursday
Nach if:2023-04-06 00:00:00


2023-04-21 00:00:00
Friday
Nach if:2023-04-24 00:00:00


2023-04-21 00:00:00
Friday
Nach if:2023-04-24 00:00:00




In [19]:
#Test Insidertrade date 2023-03-23	

insider_test_date = "2023-03-23"
insider_test_date = pd.to_datetime(insider_test_date)

#Dailychange test -> passt
amazon_test_df = amazon_df[(amazon_df.index == "2023-03-23")]

#3Daychange test -> passt
threedays_after_date = insider_test_date + BDay(3)
amazon_test_df = amazon_df[(amazon_df.index >= insider_test_date) & (amazon_df.index <= threedays_after_date)]


#3Daychange test -> passt
sevendays_after_date = insider_test_date + BDay(7)
amazon_test_df = amazon_df[(amazon_df.index >= insider_test_date) & (amazon_df.index <= sevendays_after_date)]
print(amazon_test_df)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-03-23  100.430000  101.059998   97.620003   98.709999   98.709999   
2023-03-24   98.070000   98.300003   96.400002   98.129997   98.129997   
2023-03-27   99.070000   99.339996   97.080002   98.040001   98.040001   
2023-03-28   98.110001   98.440002   96.290001   97.239998   97.239998   
2023-03-29   98.690002  100.419998   98.559998  100.250000  100.250000   
2023-03-30  101.550003  103.040001  101.010002  102.000000  102.000000   
2023-03-31  102.160004  103.489998  101.949997  103.290001  103.290001   
2023-04-03  102.300003  103.290001  101.430000  102.410004  102.410004   

              Volume Weekday_stocks Ticker  
Date                                        
2023-03-23  57559300       Thursday   AMZN  
2023-03-24  56095400         Friday   AMZN  
2023-03-27  46721300         Monday   AMZN  
2023-03-28  38720100        Tuesda

In [36]:
#Test Insidertrade date 2023-04-21	
insider_test_date = "2023-04-21"
insider_test_date = pd.to_datetime(insider_test_date)


threedays_after_date = insider_test_date + BDay(3)
amazon_test_df = amazon_df[(amazon_df.index >= insider_test_date) & (amazon_df.index <= threedays_after_date)]
print(amazon_test_df)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-04-21  106.099998  108.150002  105.080002  106.959999  106.959999   
2023-04-24  107.660004  109.230003  105.070000  106.209999  106.209999   
2023-04-25  104.910004  105.449997  102.449997  102.570000  102.570000   
2023-04-26  105.040001  106.620003  104.099998  104.980003  104.980003   

              Volume Weekday_stocks Ticker  
Date                                        
2023-04-21  86774200         Friday   AMZN  
2023-04-24  69575600         Monday   AMZN  
2023-04-25  65026800        Tuesday   AMZN  
2023-04-26  73803800      Wednesday   AMZN  
